In [1]:
import pandas as pd
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

## Загружаем данные из файла
## Данные за 01.03.22 - 15.05.22

In [2]:
data=pd.read_csv('16.05.22 full_version_evname_json_date.csv')
pd.set_option('display.max_rows', 10) # None
pd.set_option('display.max_columns', 50) # None
data.head()

,app_version_name,event_name,event_json,event_datetime
0,1.4.1,app_start,"{""balance"":""450"",""days_in_game"":""0"",""PuzzleCom...",2022-04-21 12:58:04
1,1.4.1,balance_change,"{""Currency"":""coins"",""Balance"":""450"",""balance"":...",2022-04-21 12:58:03
2,1.4.1,game_puzzle_click,"{""category_id"":""13"",""puzzle_id"":""38"",""Order_nu...",2022-04-21 12:58:45
3,1.4.1,game_puzzle_pause,"{""puzzle_id"":""38"",""Order_number"":""38"",""categor...",2022-04-21 12:59:03
4,1.4.1,game_puzzle_receive,"{""place"":""daily"",""category_id"":""13"",""Order_num...",2022-04-21 12:58:47


## Оставляем в датафрейме только event_name связанные с пазлами:  
'show_image'  
'game_puzzle_start'  
'game_puzzle_complete'  
'game_puzzle_quit'  
'game_puzzle_concat'  
'game_puzzle_receive'  
'game_puzzle_click'  
'game_puzzle_start_continue'  
'game_puzze_onscreen'  
## В jsone осталяем нужное и делаем один датафрейм

In [3]:
data=data[data['event_name'].isin(\
    ['show_image', 'game_puzzle_start', 'game_puzzle_complete', 'game_puzzle_quit', 'game_puzzle_concat', 'game_puzzle_receive', 'game_puzzle_click', 'game_puzzle_start_continue', 'game_puzze_onscreen']\
        )].reset_index(drop=True)

# развернем json
data_json = pd.json_normalize(list(map(json.loads, data.event_json)))

# сформируем фрейм с которым будем работать
data_work = data[['event_name', 'event_datetime', 'app_version_name']]

# добавим столбец с единым идентификатором пазла
data_work['cat_puzzle_id'] = data_json.category_id.astype(str) + '_' + data_json.puzzle_id.astype(str)

# сконкатим фреймы data и json
data_work = pd.concat([data_work, data_json[[\
    'category_id','place','days_in_game','PuzzleComplete','PuzzleStart','Rotate','Difficulty','completed','background','Progress','Continue','started','available','Percent','Concat','Placed']\
        ]], axis=1)

# Преобразуем типы данных
data_work['PuzzleComplete'] =pd.to_numeric(data_work.PuzzleComplete)
data_work['PuzzleStart'] =pd.to_numeric(data_work.PuzzleStart)

## Датафрейм, со всем необходимым

In [4]:
data_work.head(3)

,event_name,event_datetime,app_version_name,cat_puzzle_id,category_id,place,days_in_game,PuzzleComplete,PuzzleStart,Rotate,Difficulty,completed,background,Progress,Continue,started,available,Percent,Concat,Placed
0,game_puzzle_click,2022-04-21 12:58:45,1.4.1,13_38,13,category,0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,game_puzzle_receive,2022-04-21 12:58:47,1.4.1,13_38,13,daily,0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,show_image,2022-04-21 09:34:57,1.4.1,1_100,1,category,0,0.0,1.0,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN


## Любопытно посомтреть как изменятся значения конверсий если оставить place = daily or category

In [5]:
#data_work = data_work[data_work['place'].isin(['daily', 'catalog'])]
data_work['place'].value_counts()

daily            2916492
category         1679385
in_progress       298355
inprogress        197248
free_category     120504
completed          93978
none               19117
Name: place, dtype: int64

## Опишем функцию которая посчитает для переданого датафрейма:
Показы  
Старты  
Комплиты  
Конверсию показов в старты  
Конверсию стартов в комплиты  
Конверсию стартов в quitы  
__Все значения считаются для событий из daily, catalog и все остальные, для не собранных ранее пазлов (completed !="1") , заданного набора Difficulty, по заданным версиям__

In [6]:
def conversion_count_simply(data, mask, difficulty=['36','49','144','256','441'], version=['1.3.33', '1.4', '1.4.1', '1.4.3']):

    # будем работать только с заданными версиями приложения
    data = data[data['app_version_name'].isin(version)]
    
    # считаем столбец показов
    show_image = data.query('event_name=="show_image" and completed !="1"').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    show_image.rename(columns={'event_name':'shows_'+mask}, inplace=True)
    show_image.reset_index(inplace=True)

    # будем работать только с заданными сложностями
    data = data[data['Difficulty'].isin(difficulty)]

    # старты пазлов со сложностью difficulty
    game_puzzle_start = data.query('event_name=="game_puzzle_start"').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_start.rename(columns={'event_name':'starts_'+mask}, inplace=True)
    game_puzzle_start.reset_index(inplace=True)

    # комплиты пазлов со сложнотью difficulty
    game_puzzle_complete = data.query('event_name=="game_puzzle_complete"').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_complete.rename(columns={'event_name':'complets_'+mask}, inplace=True)
    game_puzzle_complete.reset_index(inplace=True)

    # quitы пазлов со сложностью difficulty
    game_puzzle_quit = data.query('event_name=="game_puzzle_quit"').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_quit.rename(columns={'event_name':'quits_'+mask}, inplace=True)
    game_puzzle_quit.reset_index(inplace=True)

    # удалим странный пазл
    show_image = show_image.query('cat_puzzle_id != "23_38"').reset_index(drop=True)

    # объединим в один полученные фреймы
    data_count_all = show_image.merge(game_puzzle_start, how='left').merge(game_puzzle_complete, how='left').merge(game_puzzle_quit, how='left')

    data_count_all = data_count_all.fillna(0)

    # посчитаем конверсии
    data_count_all['conv_to_start_'+mask] = round (data_count_all['starts_'+mask] / data_count_all['shows_'+mask] * 100, 2)
    data_count_all['conv_to_quit_'+mask] = round (data_count_all['quits_'+mask] / data_count_all['starts_'+mask] * 100, 2)
    data_count_all['conv_to_complete_'+mask] = round (data_count_all['complets_'+mask] / data_count_all['starts_'+mask] * 100, 2)

    # уберем нули и бесконечности
    data_count_all = data_count_all.fillna(0)
    data_count_all = data_count_all.replace(np.inf, 0)

    data_count_all['conv_to_complete_'+mask] = np.where((data_count_all['conv_to_complete_'+mask] >= 100), 99.99, data_count_all['conv_to_complete_'+mask])

    # интервальная оценка по методу Вальда
    temp_frame = pd.DataFrame()
    temp_frame['s'] = 1.96 * np.sqrt( (data_count_all['conv_to_complete_'+mask] * (100 - data_count_all['conv_to_complete_'+mask])) / (data_count_all['starts_'+mask] +0.00001)) 
    data_count_all['low_'+mask] = round( data_count_all['conv_to_complete_'+mask] - temp_frame['s'], 2)
    data_count_all['high_'+mask] = round(data_count_all['conv_to_complete_'+mask] + temp_frame['s'], 2)
    
    # интервальная оценка по преобразованию Фишера
    #temp_frame = pd.DataFrame()
    #temp_frame['fi'] = 2 * np.arcsin( np.sqrt(0.01 + data_count_all['conv_to_complete_'+mask]/100  ) )
    #data_count_all['low_'+mask] = round(100* np.sin ( ( temp_frame['fi'] - 1.96*(1/np.sqrt(data_count_all['starts_'+mask]+0.000001)) )/2 )**2, 2)
    #data_count_all['high_'+mask] = round(100*  np.sin ( ( temp_frame['fi'] + 1.96*(1/np.sqrt(data_count_all['starts_'+mask]+0.000001)) )/2 )**2, 2)

    return data_count_all



## Опишем функцию которая посчитает для переданого датафрейма ПЕРВЫЕ конверсии:


In [7]:
def conversion_count_first(data, mask, difficulty=['36','49','144','256','441'], version=['1.3.33', '1.4', '1.4.1', '1.4.3']):

    # будем работать только с заданными версиями приложения
    data = data[data['app_version_name'].isin(version)]
    
    # считаем столбец показов
    show_image = data.query('event_name=="show_image" and completed !="1" and PuzzleStart == 0').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    show_image.rename(columns={'event_name':'shows_'+mask}, inplace=True)
    show_image.reset_index(inplace=True)

    # будем работать только с заданными сложностями
    data = data[data['Difficulty'].isin(difficulty)]

    # старты пазлов со сложностью difficulty
    game_puzzle_start = data.query('event_name=="game_puzzle_start" and PuzzleStart == 0').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_start.rename(columns={'event_name':'starts_'+mask}, inplace=True)
    game_puzzle_start.reset_index(inplace=True)

    # комплиты пазлов со сложнотью difficulty
    game_puzzle_complete = data.query('event_name=="game_puzzle_complete" and PuzzleComplete == 1').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_complete.rename(columns={'event_name':'complets_'+mask}, inplace=True)
    game_puzzle_complete.reset_index(inplace=True)

    # quitы пазлов со сложностью difficulty
    game_puzzle_quit = data.query('event_name=="game_puzzle_quit" and PuzzleStart == 1').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_quit.rename(columns={'event_name':'quits_'+mask}, inplace=True)
    game_puzzle_quit.reset_index(inplace=True)

    # удалим странный пазл
    show_image = show_image.query('cat_puzzle_id != "23_38"').reset_index(drop=True)

    # объединим в один полученные фреймы
    data_count_all = show_image.merge(game_puzzle_start, how='left').merge(game_puzzle_complete, how='left').merge(game_puzzle_quit, how='left')

    data_count_all = data_count_all.fillna(0)

    # посчитаем конверсии
    data_count_all['conv_to_start_'+mask] = round (data_count_all['starts_'+mask] / data_count_all['shows_'+mask] * 100, 2)
    data_count_all['conv_to_quit_'+mask] = round (data_count_all['quits_'+mask] / data_count_all['starts_'+mask] * 100, 2)
    data_count_all['conv_to_complete_'+mask] = round (data_count_all['complets_'+mask] / data_count_all['starts_'+mask] * 100, 2)

    # уберем нули и бесконечности
    data_count_all = data_count_all.fillna(0)
    data_count_all = data_count_all.replace(np.inf, 0)

    data_count_all['conv_to_complete_'+mask] = np.where((data_count_all['conv_to_complete_'+mask] >= 100), 99.99, data_count_all['conv_to_complete_'+mask])

    # интервальная оценка по методу Вальда
    temp_frame = pd.DataFrame()
    temp_frame['s'] = 1.96 * np.sqrt( (data_count_all['conv_to_complete_'+mask] * (100 - data_count_all['conv_to_complete_'+mask])) / (data_count_all['starts_'+mask] +0.00001)) 
    data_count_all['low_'+mask] = round( data_count_all['conv_to_complete_'+mask] - temp_frame['s'], 2)
    data_count_all['high_'+mask] = round(data_count_all['conv_to_complete_'+mask] + temp_frame['s'], 2)
    
    return data_count_all

## Опишем функцию которая посчитает для переданого датафрейма ПЕРВЫЕ ПЛЮС конверсии:

In [8]:
def conversion_count_first_plus(data, mask, difficulty=['36','49','144','256','441'], version=['1.3.33', '1.4', '1.4.1', '1.4.3']):

    # будем работать только с заданными версиями приложения
    data = data[data['app_version_name'].isin(version)]
    
    # считаем столбец показов
    show_image = data.query('event_name=="show_image" and completed !="1" and PuzzleStart >0').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    show_image.rename(columns={'event_name':'shows_'+mask}, inplace=True)
    show_image.reset_index(inplace=True)

    # будем работать только с заданными сложностями
    data = data[data['Difficulty'].isin(difficulty)]

    # старты пазлов со сложностью difficulty
    game_puzzle_start = data.query('event_name=="game_puzzle_start" and PuzzleStart > 0').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_start.rename(columns={'event_name':'starts_'+mask}, inplace=True)
    game_puzzle_start.reset_index(inplace=True)

    # комплиты пазлов со сложнотью difficulty
    game_puzzle_complete = data.query('event_name=="game_puzzle_complete" and PuzzleComplete > 1').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_complete.rename(columns={'event_name':'complets_'+mask}, inplace=True)
    game_puzzle_complete.reset_index(inplace=True)

    # quitы пазлов со сложностью difficulty
    game_puzzle_quit = data.query('event_name=="game_puzzle_quit" and PuzzleStart > 1').groupby(by=['cat_puzzle_id', 'category_id']).count().iloc[:,0:1]
    game_puzzle_quit.rename(columns={'event_name':'quits_'+mask}, inplace=True)
    game_puzzle_quit.reset_index(inplace=True)

    # удалим странный пазл
    show_image = show_image.query('cat_puzzle_id != "23_38"').reset_index(drop=True)

    # объединим в один полученные фреймы
    data_count_all = show_image.merge(game_puzzle_start, how='left').merge(game_puzzle_complete, how='left').merge(game_puzzle_quit, how='left')

    data_count_all = data_count_all.fillna(0)

    # посчитаем конверсии
    data_count_all['conv_to_start_'+mask] = round (data_count_all['starts_'+mask] / data_count_all['shows_'+mask] * 100, 2)
    data_count_all['conv_to_quit_'+mask] = round (data_count_all['quits_'+mask] / data_count_all['starts_'+mask] * 100, 2)
    data_count_all['conv_to_complete_'+mask] = round (data_count_all['complets_'+mask] / data_count_all['starts_'+mask] * 100, 2)

    # уберем нули и бесконечности
    data_count_all = data_count_all.fillna(0)
    data_count_all = data_count_all.replace(np.inf, 0)

    data_count_all['conv_to_complete_'+mask] = np.where((data_count_all['conv_to_complete_'+mask] >= 100), 99.99, data_count_all['conv_to_complete_'+mask])

    # интервальная оценка по методу Вальда
    temp_frame = pd.DataFrame()
    temp_frame['s'] = 1.96 * np.sqrt( (data_count_all['conv_to_complete_'+mask] * (100 - data_count_all['conv_to_complete_'+mask])) / (data_count_all['starts_'+mask] +0.00001)) 
    data_count_all['low_'+mask] = round( data_count_all['conv_to_complete_'+mask] - temp_frame['s'], 2)
    data_count_all['high_'+mask] = round(data_count_all['conv_to_complete_'+mask] + temp_frame['s'], 2)

    return data_count_all

# Расчитаем конверсии

## Итоговая таблица конверсий для всех стартов, по версиям, для всех Difficulty, по пазлам

In [9]:
# полный список версий из фрейма
versions_list = list(data_work['app_version_name'].value_counts().index)
# применяем для расчета занчений_full

In [25]:

conversion_table_simply = pd.concat([conversion_count_simply(data_work, 'full', version=versions_list),
        conversion_count_simply(data_work, '1.3.33', version=['1.3.33']).iloc[:,2:120],
            conversion_count_simply(data_work, '1.4', version=['1.4']).iloc[:,2:120],
                conversion_count_simply(data_work, '1.4.1', version=['1.4.1']).iloc[:,2:120],
                    conversion_count_simply(data_work, '1.4.3', version=['1.4.3']).iloc[:,2:120]
                        ], axis=1)

conversion_table_simply.sort_values('shows_full',ascending=False)

,cat_puzzle_id,category_id,shows_full,starts_full,complets_full,quits_full,conv_to_start_full,conv_to_quit_full,conv_to_complete_full,low_full,high_full,shows_1.3.33,starts_1.3.33,complets_1.3.33,quits_1.3.33,conv_to_start_1.3.33,conv_to_quit_1.3.33,conv_to_complete_1.3.33,low_1.3.33,high_1.3.33,shows_1.4,starts_1.4,complets_1.4,quits_1.4,conv_to_start_1.4,conv_to_quit_1.4,conv_to_complete_1.4,low_1.4,high_1.4,shows_1.4.1,starts_1.4.1,complets_1.4.1,quits_1.4.1,conv_to_start_1.4.1,conv_to_quit_1.4.1,conv_to_complete_1.4.1,low_1.4.1,high_1.4.1,shows_1.4.3,starts_1.4.3,complets_1.4.3,quits_1.4.3,conv_to_start_1.4.3,conv_to_quit_1.4.3,conv_to_complete_1.4.3,low_1.4.3,high_1.4.3
371,1_33,1,93021,4913,3875,715.0,5.28,14.55,78.87,77.73,80.01,519,14.0,11.0,2.0,2.70,14.29,78.57,57.08,100.06,31613,1540,1152,191.0,4.87,12.40,74.81,72.64,76.98,21858,1250,1002,202.0,5.72,16.16,80.16,77.95,82.37,12138,559,436.0,102.0,4.61,18.25,78.00,74.57,81.43
873,31_8,31,83151,4319,3315,663.0,5.19,15.35,76.75,75.49,78.01,487,25.0,14.0,3.0,5.13,12.00,56.00,36.54,75.46,32632,1522,1084,192.0,4.66,12.61,71.22,68.95,73.49,21782,1253,982,260.0,5.75,20.75,78.37,76.09,80.65,460,4,11.0,2.0,0.87,50.00,99.99,99.01,100.97
964,34_1,34,82315,4131,3091,790.0,5.02,19.12,74.82,73.50,76.14,442,14.0,10.0,1.0,3.17,7.14,71.43,47.77,95.09,31409,1368,961,223.0,4.36,16.30,70.25,67.83,72.67,22151,1196,926,260.0,5.40,21.74,77.42,75.05,79.79,516,14,22.0,6.0,2.71,42.86,99.99,99.47,100.51
496,21_3,21,75834,4823,3470,904.0,6.36,18.74,71.95,70.68,73.22,475,23.0,14.0,0.0,4.84,0.00,60.87,40.92,80.82,28837,1762,1176,280.0,6.11,15.89,66.74,64.54,68.94,20332,1398,1031,308.0,6.88,22.03,73.75,71.44,76.06,374,5,8.0,2.0,1.34,40.00,99.99,99.11,100.87
815,31_27,31,16964,709,601,92.0,4.18,12.98,84.77,82.13,87.41,43,2.0,2.0,0.0,4.65,0.00,99.99,98.60,101.38,1039,64,56,5.0,6.16,7.81,87.50,79.40,95.60,1116,70,67,7.0,6.27,10.00,95.71,90.96,100.46,13247,453,366.0,62.0,3.42,13.69,80.79,77.16,84.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290,7_69,7,878,41,43,2.0,4.67,4.88,99.99,99.68,100.30,25,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,253,12,13,1.0,4.74,8.33,99.99,99.42,100.56,188,7,8,0.0,3.72,0.00,99.99,99.25,100.73,43,6,6.0,1.0,13.95,16.67,99.99,99.19,100.79
1233,7_17,7,869,49,45,9.0,5.64,18.37,91.84,84.17,99.51,18,1.0,1.0,0.0,5.56,0.00,99.99,98.03,101.95,252,8,9,1.0,3.17,12.50,99.99,99.30,100.68,242,7,7,1.0,2.89,14.29,99.99,99.25,100.73,127,10,9.0,3.0,7.87,30.00,90.00,71.41,108.59
555,21_83,21,865,73,59,11.0,8.44,15.07,80.82,71.79,89.85,23,1.0,1.0,0.0,4.35,0.00,99.99,98.03,101.95,214,21,17,0.0,9.81,0.00,80.95,64.15,97.75,194,19,13,2.0,9.79,10.53,68.42,47.52,89.32,94,4,4.0,0.0,4.26,0.00,99.99,99.01,100.97
1287,7_66,7,863,41,40,6.0,4.75,14.63,97.56,92.84,102.28,26,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,222,12,9,4.0,5.41,33.33,75.00,50.50,99.50,218,9,9,2.0,4.13,22.22,99.99,99.34,100.64,84,3,4.0,0.0,3.57,0.00,99.99,98.86,101.12


## Итоговая таблица конверсий для ПЕРВЫХ стартов, по версиям, для всех Difficulty = 36, по пазлам

In [26]:
diff = ['36']
conversion_table_first = pd.concat([conversion_count_first(data_work, 'full', difficulty=diff, version=versions_list),
        conversion_count_first(data_work, '1.3.33', version=['1.3.33'], difficulty=diff).iloc[:,2:120],
            conversion_count_first(data_work, '1.4', version=['1.4'], difficulty=diff).iloc[:,2:120],
                conversion_count_first(data_work, '1.4.1', version=['1.4.1'], difficulty=diff).iloc[:,2:120],
                    conversion_count_first(data_work, '1.4.3', version=['1.4.3'], difficulty=diff).iloc[:,2:120]
                        ], axis=1)
conversion_table_first.sort_values('shows_full',ascending=False)

,cat_puzzle_id,category_id,shows_full,starts_full,complets_full,quits_full,conv_to_start_full,conv_to_quit_full,conv_to_complete_full,low_full,high_full,shows_1.3.33,starts_1.3.33,complets_1.3.33,quits_1.3.33,conv_to_start_1.3.33,conv_to_quit_1.3.33,conv_to_complete_1.3.33,low_1.3.33,high_1.3.33,shows_1.4,starts_1.4,complets_1.4,quits_1.4,conv_to_start_1.4,conv_to_quit_1.4,conv_to_complete_1.4,low_1.4,high_1.4,shows_1.4.1,starts_1.4.1,complets_1.4.1,quits_1.4.1,conv_to_start_1.4.1,conv_to_quit_1.4.1,conv_to_complete_1.4.1,low_1.4.1,high_1.4.1,shows_1.4.3,starts_1.4.3,complets_1.4.3,quits_1.4.3,conv_to_start_1.4.3,conv_to_quit_1.4.3,conv_to_complete_1.4.3,low_1.4.3,high_1.4.3
371,1_33,1,22579,611.0,427.0,88.0,2.71,14.40,69.89,66.25,73.53,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6329,152.0,100.0,16.0,2.40,10.53,65.79,58.25,73.33,5998,176.0,114.0,34.0,2.93,19.32,64.77,57.71,71.83,2881,112.0,76.0,13.0,3.89,11.61,67.86,59.21,76.51
873,31_8,31,20351,889.0,666.0,110.0,4.37,12.37,74.92,72.07,77.77,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6515,283.0,205.0,27.0,4.34,9.54,72.44,67.23,77.65,6277,295.0,212.0,55.0,4.70,18.64,71.86,66.73,76.99,12,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
964,34_1,34,19923,820.0,583.0,126.0,4.12,15.37,71.10,68.00,74.20,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6387,254.0,169.0,30.0,3.98,11.81,66.54,60.74,72.34,6101,266.0,176.0,58.0,4.36,21.80,66.17,60.48,71.86,30,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
496,21_3,21,19027,1314.0,849.0,193.0,6.91,14.69,64.61,62.02,67.20,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6143,482.0,299.0,75.0,7.85,15.56,62.03,57.70,66.36,5784,411.0,253.0,71.0,7.11,17.27,61.56,56.86,66.26,10,0.0,1.0,0.0,0.00,0.00,0.00,0.00,0.00
405,1_64,1,3507,972.0,712.0,142.0,27.72,14.61,73.25,70.47,76.03,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,192,5.0,5.0,1.0,2.60,20.00,99.99,99.11,100.87,131,2.0,1.0,0.0,1.53,0.00,50.00,-19.30,119.30,2923,962.0,702.0,141.0,32.91,14.66,72.97,70.16,75.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,7_73,7,76,1.0,1.0,1.0,1.32,100.00,99.99,98.03,101.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,1.0,1.0,1.0,5.88,100.00,99.99,98.03,101.95,16,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,11,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
1288,7_67,7,74,4.0,3.0,1.0,5.41,25.00,75.00,32.56,117.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,1.0,1.0,0.0,4.76,0.00,99.99,98.03,101.95,21,3.0,2.0,1.0,14.29,33.33,66.67,13.33,120.01,10,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
1278,7_58,7,74,3.0,1.0,1.0,4.05,33.33,33.33,-20.01,86.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,1.0,1.0,0.0,5.00,0.00,99.99,98.03,101.95,20,1.0,0.0,0.0,5.00,0.00,0.00,0.00,0.00,7,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
1266,7_47,7,72,4.0,4.0,0.0,5.56,0.00,99.99,99.01,100.97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,2.0,2.0,0.0,14.29,0.00,99.99,98.60,101.38,22,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,11,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00


## Итоговая таблица конверсий для первых ПЛЮС стартов, по версиям, для всех Difficulty = 36, по пазлам

In [27]:
diff = ['36']
conversion_table_first_plus = pd.concat([conversion_count_first_plus(data_work, 'full', difficulty=diff, version=versions_list),
        conversion_count_first_plus(data_work, '1.3.33', version=['1.3.33'], difficulty=diff).iloc[:,2:120],
            conversion_count_first_plus(data_work, '1.4', version=['1.4'], difficulty=diff).iloc[:,2:120],
                conversion_count_first_plus(data_work, '1.4.1', version=['1.4.1'], difficulty=diff).iloc[:,2:120],
                    conversion_count_first_plus(data_work, '1.4.3', version=['1.4.3'], difficulty=diff).iloc[:,2:120]
                        ], axis=1)
conversion_table_first_plus.sort_values('shows_full',ascending=False)

,cat_puzzle_id,category_id,shows_full,starts_full,complets_full,quits_full,conv_to_start_full,conv_to_quit_full,conv_to_complete_full,low_full,high_full,shows_1.3.33,starts_1.3.33,complets_1.3.33,quits_1.3.33,conv_to_start_1.3.33,conv_to_quit_1.3.33,conv_to_complete_1.3.33,low_1.3.33,high_1.3.33,shows_1.4,starts_1.4,complets_1.4,quits_1.4,conv_to_start_1.4,conv_to_quit_1.4,conv_to_complete_1.4,low_1.4,high_1.4,shows_1.4.1,starts_1.4.1,complets_1.4.1,quits_1.4.1,conv_to_start_1.4.1,conv_to_quit_1.4.1,conv_to_complete_1.4.1,low_1.4.1,high_1.4.1,shows_1.4.3,starts_1.4.3,complets_1.4.3,quits_1.4.3,conv_to_start_1.4.3,conv_to_quit_1.4.3,conv_to_complete_1.4.3,low_1.4.3,high_1.4.3
371,1_33,1,70418,3111,2707,294.0,4.42,9.45,87.01,85.83,88.19,353,7.0,7.0,0.0,1.98,0.0,99.99,99.25,100.73,25284,1008,839,78.0,3.99,7.74,83.23,80.92,85.54,15860,772,698,85.0,4.87,11.01,90.41,88.33,92.49,9257,313.0,272.0,41.0,3.38,13.10,86.90,83.16,90.64
873,31_8,31,62791,2250,1972,238.0,3.58,10.58,87.64,86.28,89.00,321,11.0,8.0,0.0,3.43,0.0,72.73,46.41,99.05,26117,808,656,80.0,3.09,9.90,81.19,78.50,83.88,15505,647,590,87.0,4.17,13.45,91.19,89.01,93.37,448,0.0,6.0,0.0,0.00,0.00,0.00,0.00,0.00
964,34_1,34,62382,2046,1796,244.0,3.28,11.93,87.78,86.36,89.20,280,6.0,5.0,0.0,2.14,0.0,83.33,53.51,113.15,25022,682,567,78.0,2.73,11.44,83.14,80.33,85.95,16050,601,551,89.0,3.74,14.81,91.68,89.47,93.89,486,6.0,11.0,1.0,1.23,16.67,99.99,99.19,100.79
496,21_3,21,56806,2330,1966,320.0,4.10,13.73,84.38,82.91,85.85,316,6.0,4.0,0.0,1.90,0.0,66.67,28.95,104.39,22694,839,659,92.0,3.70,10.97,78.55,75.77,81.33,14548,671,588,110.0,4.61,16.39,87.63,85.14,90.12,364,3.0,4.0,0.0,0.82,0.00,99.99,98.86,101.12
815,31_27,31,13626,375,332,25.0,2.75,6.67,88.53,85.30,91.76,41,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,935,26,23,0.0,2.78,0.00,88.46,76.18,100.74,1014,34,33,0.0,3.35,0.00,97.06,91.38,102.74,10227,255.0,217.0,24.0,2.49,9.41,85.10,80.73,89.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,7_78,7,792,25,22,2.0,3.16,8.00,88.00,75.26,100.74,14,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,268,6,4,1.0,2.24,16.67,66.67,28.95,104.39,134,5,5,0.0,3.73,0.00,99.99,99.11,100.87,78,3.0,3.0,0.0,3.85,0.00,99.99,98.86,101.12
1233,7_17,7,790,27,24,2.0,3.42,7.41,88.89,77.04,100.74,16,1.0,1.0,0.0,6.25,0.0,99.99,98.03,101.95,231,5,5,1.0,2.16,20.00,99.99,99.11,100.87,225,4,4,0.0,1.78,0.00,99.99,99.01,100.97,121,6.0,6.0,0.0,4.96,0.00,99.99,99.19,100.79
1287,7_66,7,781,18,18,6.0,2.30,33.33,99.99,99.53,100.45,22,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,208,5,3,4.0,2.40,80.00,60.00,17.06,102.94,191,4,5,2.0,2.09,50.00,99.99,99.01,100.97,75,2.0,2.0,0.0,2.67,0.00,99.99,98.60,101.38
555,21_83,21,761,41,36,7.0,5.39,17.07,87.80,77.78,97.82,23,1.0,1.0,0.0,4.35,0.0,99.99,98.03,101.95,183,9,8,0.0,4.92,0.00,88.89,68.36,109.42,165,11,8,1.0,6.67,9.09,72.73,46.41,99.05,85,2.0,2.0,0.0,2.35,0.00,99.99,98.60,101.38


## Выведем средние и доверительные интервалы для средних по всем конверсиям и версиям в одну таблицу.  
### (starts hits, % - процент наблюдений стартов приходящихся на версию приложения)

In [28]:
conv_list = ['conv_to_start_full','conv_to_complete_full','conv_to_quit_full',\
    'conv_to_start_1.3.33','conv_to_complete_1.3.33','conv_to_quit_1.3.33',\
        'conv_to_start_1.4','conv_to_complete_1.4','conv_to_quit_1.4',\
           'conv_to_start_1.4.1','conv_to_complete_1.4.1','conv_to_quit_1.4.1',\
               'conv_to_start_1.4.3','conv_to_complete_1.4.3','conv_to_quit_1.4.3']

In [29]:
import scipy.stats as st

In [30]:
conversion_only = conversion_table_simply[conv_list]

conv_matrix_simply = pd.DataFrame(columns=['full', 'v1.3.33', 'v1.4', 'v1.4.1', 'v1.4.3'], index = ['conv_to_start', 'conv_to_complete', 'conv_to_quit'])
counter = 0
for i in range(5):
    t=[]
    
    for j in range(3):
        x= []
        y=[]
        y = np.round(list(st.t.interval(0.95, len(conversion_only.iloc[:,counter])-1, loc=np.mean(conversion_only.iloc[:,counter]), scale=st.sem(conversion_only.iloc[:,counter]))),2)
        x.append(round(conversion_only.iloc[:,counter].mean(),2))
        x.append(list(y))
        counter+=1
        t.append(x)
    conv_matrix_simply.iloc[:,i] = t
conv_matrix_simply

,full,v1.3.33,v1.4,v1.4.1,v1.4.3
conv_to_start,"[4.98, [4.91, 5.05]]","[3.71, [3.5, 3.92]]","[4.56, [4.48, 4.65]]","[5.32, [5.22, 5.42]]","[4.26, [4.15, 4.37]]"
conv_to_complete,"[86.93, [86.62, 87.23]]","[60.93, [58.42, 63.44]]","[79.47, [78.87, 80.06]]","[88.94, [88.42, 89.46]]","[90.77, [90.08, 91.45]]"
conv_to_quit,"[12.22, [11.9, 12.53]]","[1.84, [1.23, 2.45]]","[11.73, [11.16, 12.3]]","[13.86, [13.18, 14.53]]","[11.82, [10.69, 12.94]]"


In [31]:
conversion_only = conversion_table_first[conv_list]

conv_matrix_first = pd.DataFrame(columns=['full', 'v1.3.33', 'v1.4', 'v1.4.1', 'v1.4.3'], index = ['conv_to_start_first', 'conv_to_complete_first', 'conv_to_quit_first'])
counter = 0
for i in range(5):
    t=[]
    for j in range(3):
        x= []
        y=[]
        y = np.round(list(st.t.interval(0.95, len(conversion_only.iloc[:,counter])-1, loc=np.mean(conversion_only.iloc[:,counter]), scale=st.sem(conversion_only.iloc[:,counter]))),2)
        x.append(round(conversion_only.iloc[:,counter].mean(),2))
        x.append(list(y))
        counter+=1
        t.append(x)
    conv_matrix_first.iloc[:,i] = t
conv_matrix_first

,full,v1.3.33,v1.4,v1.4.1,v1.4.3
conv_to_start_first,"[3.53, [3.41, 3.64]]","[1.47, [nan, nan]]","[4.23, [4.05, 4.41]]","[4.14, [3.95, 4.33]]","[0.81, [0.67, 0.95]]"
conv_to_complete_first,"[71.58, [70.23, 72.92]]","[3.18, [nan, nan]]","[58.52, [56.37, 60.68]]","[54.95, [52.69, 57.21]]","[9.93, [8.37, 11.49]]"
conv_to_quit_first,"[12.77, [11.88, 13.65]]","[0.32, [nan, nan]]","[10.59, [9.39, 11.8]]","[11.72, [10.2, 13.24]]","[2.17, [1.45, 2.89]]"


In [32]:
conversion_only = conversion_table_first_plus[conv_list]

conv_matrix_first_plus = pd.DataFrame(columns=['full', 'v1.3.33', 'v1.4', 'v1.4.1', 'v1.4.3'], index = ['conv_to_start_first_plus', 'conv_to_complete_first_plus', 'conv_to_quit_first_plus'])
counter = 0
for i in range(5):
    t=[]
    for j in range(3):
        x= []
        y=[]
        y = np.round(list(st.t.interval(0.95, len(conversion_only.iloc[:,counter])-1, loc=np.mean(conversion_only.iloc[:,counter]), scale=st.sem(conversion_only.iloc[:,counter]))),2)
        x.append(round(conversion_only.iloc[:,counter].mean(),2))
        x.append(list(y))
        counter+=1
        t.append(x)
    conv_matrix_first_plus.iloc[:,i] = t
conv_matrix_first_plus

,full,v1.3.33,v1.4,v1.4.1,v1.4.3
conv_to_start_first_plus,"[2.81, [2.77, 2.86]]","[1.55, [1.4, 1.69]]","[2.44, [2.38, 2.5]]","[3.31, [3.24, 3.38]]","[2.55, [2.47, 2.63]]"
conv_to_complete_first_plus,"[92.83, [92.54, 93.12]]","[32.48, [29.98, 34.97]]","[85.54, [84.83, 86.26]]","[94.07, [93.62, 94.53]]","[89.11, [87.83, 90.39]]"
conv_to_quit_first_plus,"[6.76, [6.48, 7.04]]","[0.36, [0.07, 0.65]]","[7.13, [6.6, 7.66]]","[8.33, [7.68, 8.97]]","[6.37, [5.38, 7.37]]"


In [33]:
result_matrix = pd.concat([conv_matrix_simply, conv_matrix_first, conv_matrix_first_plus])
start_hits_list = ['starts_full', 'starts_1.3.33' ,'starts_1.4' ,'starts_1.4.1','starts_1.4.3']

start_hits=[]
for i in start_hits_list:
    start_hits.append(round(conversion_table_simply[i].sum()*100/conversion_table_simply[start_hits_list[0]].sum(),1))
result_matrix.loc['starts hits, %'] = start_hits
result_matrix

,full,v1.3.33,v1.4,v1.4.1,v1.4.3
conv_to_start,"[4.98, [4.91, 5.05]]","[3.71, [3.5, 3.92]]","[4.56, [4.48, 4.65]]","[5.32, [5.22, 5.42]]","[4.26, [4.15, 4.37]]"
conv_to_complete,"[86.93, [86.62, 87.23]]","[60.93, [58.42, 63.44]]","[79.47, [78.87, 80.06]]","[88.94, [88.42, 89.46]]","[90.77, [90.08, 91.45]]"
conv_to_quit,"[12.22, [11.9, 12.53]]","[1.84, [1.23, 2.45]]","[11.73, [11.16, 12.3]]","[13.86, [13.18, 14.53]]","[11.82, [10.69, 12.94]]"
conv_to_start_first,"[3.53, [3.41, 3.64]]","[1.47, [nan, nan]]","[4.23, [4.05, 4.41]]","[4.14, [3.95, 4.33]]","[0.81, [0.67, 0.95]]"
conv_to_complete_first,"[71.58, [70.23, 72.92]]","[3.18, [nan, nan]]","[58.52, [56.37, 60.68]]","[54.95, [52.69, 57.21]]","[9.93, [8.37, 11.49]]"
conv_to_quit_first,"[12.77, [11.88, 13.65]]","[0.32, [nan, nan]]","[10.59, [9.39, 11.8]]","[11.72, [10.2, 13.24]]","[2.17, [1.45, 2.89]]"
conv_to_start_first_plus,"[2.81, [2.77, 2.86]]","[1.55, [1.4, 1.69]]","[2.44, [2.38, 2.5]]","[3.31, [3.24, 3.38]]","[2.55, [2.47, 2.63]]"
conv_to_complete_first_plus,"[92.83, [92.54, 93.12]]","[32.48, [29.98, 34.97]]","[85.54, [84.83, 86.26]]","[94.07, [93.62, 94.53]]","[89.11, [87.83, 90.39]]"
conv_to_quit_first_plus,"[6.76, [6.48, 7.04]]","[0.36, [0.07, 0.65]]","[7.13, [6.6, 7.66]]","[8.33, [7.68, 8.97]]","[6.37, [5.38, 7.37]]"
"starts hits, %",100.0,1.1,26.6,23.8,11.1


## Таблица конверсий разных сложностей (без разбивки по версиям)

In [34]:
dif_ty=['36','49','144','256','441']

conversion_table_difficulty = pd.concat([conversion_count_simply(data_work, 'full', difficulty=dif_ty, version=versions_list),\
    conversion_count_simply(data_work, '36', ['36'], version=versions_list).iloc[:,2:120],\
        conversion_count_simply(data_work, '49', ['49'], version=versions_list).iloc[:,2:120],\
            conversion_count_simply(data_work, '144', ['144'], version=versions_list).iloc[:,2:120],\
                conversion_count_simply(data_work, '256', ['256'], version=versions_list).iloc[:,2:120],\
                    conversion_count_simply(data_work, '441', ['441'], version=versions_list).iloc[:,2:120]], axis=1)

conversion_table_difficulty.sort_values('shows_full', ascending=False).head()

,cat_puzzle_id,category_id,shows_full,starts_full,complets_full,quits_full,conv_to_start_full,conv_to_quit_full,conv_to_complete_full,low_full,high_full,shows_36,starts_36,complets_36,quits_36,conv_to_start_36,conv_to_quit_36,conv_to_complete_36,low_36,high_36,shows_49,starts_49,complets_49,quits_49,conv_to_start_49,...,complets_144,quits_144,conv_to_start_144,conv_to_quit_144,conv_to_complete_144,low_144,high_144,shows_256,starts_256,complets_256,quits_256,conv_to_start_256,conv_to_quit_256,conv_to_complete_256,low_256,high_256,shows_441,starts_441,complets_441,quits_441,conv_to_start_441,conv_to_quit_441,conv_to_complete_441,low_441,high_441
371,1_33,1,93021,4913,3875,715.0,5.28,14.55,78.87,77.73,80.01,93021,3722,3134,382.0,4.00,10.26,84.20,83.03,85.37,93021,768,639,122.0,0.83,...,86.0,122.0,0.30,43.88,30.94,25.51,36.37,93021,54.0,13.0,24.0,0.06,44.44,24.07,12.67,35.47,93021,91.0,3.0,65.0,0.10,71.43,3.30,-0.37,6.97
873,31_8,31,83151,4319,3315,663.0,5.19,15.35,76.75,75.49,78.01,83151,3139,2638,348.0,3.78,11.09,84.04,82.76,85.32,83151,697,567,116.0,0.84,...,94.0,86.0,0.33,31.27,34.18,28.57,39.79,83151,92.0,12.0,54.0,0.11,58.70,13.04,6.16,19.92,83151,116.0,4.0,59.0,0.14,50.86,3.45,0.13,6.77
964,34_1,34,82315,4131,3091,790.0,5.02,19.12,74.82,73.50,76.14,82315,2866,2379,370.0,3.48,12.91,83.01,81.64,84.38,82315,725,607,137.0,0.88,...,88.0,148.0,0.35,51.57,30.66,25.33,35.99,82315,89.0,14.0,50.0,0.11,56.18,15.73,8.17,23.29,82315,164.0,3.0,85.0,0.20,51.83,1.83,-0.22,3.88
496,21_3,21,75834,4823,3470,904.0,6.36,18.74,71.95,70.68,73.22,75834,3644,2815,513.0,4.81,14.08,77.25,75.89,78.61,75834,735,557,148.0,0.97,...,83.0,127.0,0.34,49.42,32.30,26.58,38.02,75834,68.0,11.0,54.0,0.09,79.41,16.18,7.43,24.93,75834,119.0,4.0,62.0,0.16,52.10,3.36,0.12,6.60
815,31_27,31,16964,709,601,92.0,4.18,12.98,84.77,82.13,87.41,16964,416,366,27.0,2.45,6.49,87.98,84.85,91.11,16964,189,172,18.0,1.11,...,57.0,36.0,0.46,46.15,73.08,63.24,82.92,16964,13.0,5.0,6.0,0.08,46.15,38.46,12.01,64.91,16964,13.0,1.0,5.0,0.08,38.46,7.69,-6.79,22.17


In [35]:
conversion_only = conversion_table_difficulty[['conv_to_start_full','conv_to_complete_full','conv_to_quit_full',\
    'conv_to_start_36','conv_to_complete_36','conv_to_quit_36',\
        'conv_to_start_49','conv_to_complete_49','conv_to_quit_49',\
           'conv_to_start_144','conv_to_complete_144','conv_to_quit_144',\
               'conv_to_start_256','conv_to_complete_256','conv_to_quit_256',\
                   'conv_to_start_441','conv_to_complete_441','conv_to_quit_441']]

In [36]:
conv_matrix_difficulty = pd.DataFrame(columns=['full', '36', '49', '144', '256', '441'], index = ['conv_to_start_difficulty', 'conv_to_complete_difficulty', 'conv_to_quit_difficulty'])
counter = 0
for i in range(6):
    x= []
    for j in range(3):
        x.append(round(conversion_only.iloc[:,counter].mean(),2))
        counter+=1
    conv_matrix_difficulty.iloc[:,i] = x
conv_matrix_difficulty

start_hits_list = ['starts_full', 'starts_36' ,'starts_49' ,'starts_144','starts_256', 'starts_441']
start_hits=[]
for i in start_hits_list:
    start_hits.append(round(conversion_table_difficulty[i].sum()*100/conversion_table_difficulty[start_hits_list[0]].sum(),1))
conv_matrix_difficulty.loc['starts hits, %'] = start_hits
conv_matrix_difficulty

,full,36,49,144,256,441
conv_to_start_difficulty,4.98,2.88,1.48,0.45,0.10,0.08
conv_to_complete_difficulty,86.93,90.51,91.70,64.94,44.68,11.91
conv_to_quit_difficulty,12.22,7.48,8.64,36.35,44.75,48.56
"starts hits, %",100.00,59.80,27.70,8.70,2.00,1.80


## Сохранимся в excel файл

In [37]:
with pd.ExcelWriter("17.05.22 Conversions_plus_confint.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    result_matrix.to_excel(writer, sheet_name="Средние конверсий", index=False)
    conv_matrix_difficulty.to_excel(writer, sheet_name="Средние по сложностям", index=False)
    conversion_table_simply.sort_values('shows_full', ascending=False).to_excel(writer, sheet_name="Полные конверсии", index=False)
    conversion_table_first.sort_values('shows_full', ascending=False).to_excel(writer, sheet_name="Первый старт", index=False)
    conversion_table_first_plus.sort_values('shows_full', ascending=False).to_excel(writer, sheet_name="Последующие старты", index=False)
    conversion_table_difficulty.sort_values('shows_full', ascending=False).to_excel(writer, sheet_name="Сложности", index=False)

## Обратим внимание на странное значение Конверсии первого старта в копмлит с версией 1.4.3

In [23]:
first_starts_143 = data_work[data_work['app_version_name'] == '1.4.3'].query('event_name == "game_puzzle_complete"')
print('Количество первых комплитов ', first_starts_143.query('PuzzleComplete == 1').groupby(by='cat_puzzle_id').count().shape[0])

first_complets_143 = data_work[data_work['app_version_name'] == '1.4.3'].query('event_name == "game_puzzle_start"')
print('Количество первых стартов ',first_complets_143.query('PuzzleStart ==0').groupby(by='cat_puzzle_id').count().shape[0])

Количество первых комплитов  187
Количество первых стартов  251


Например такие же цифры для версии 1.4 будут выглядеть следующим образом

In [24]:
first_starts_14 = data_work[data_work['app_version_name'] == '1.4'].query('event_name == "game_puzzle_complete"')
print('Количество первых комплитов ', first_starts_14.query('PuzzleComplete == 1').groupby(by='cat_puzzle_id').count().shape[0])

first_complets_14 = data_work[data_work['app_version_name'] == '1.4'].query('event_name == "game_puzzle_start"')
print('Количество первых стартов ',first_complets_14.query('PuzzleStart ==0').groupby(by='cat_puzzle_id').count().shape[0])


Количество первых комплитов  1054
Количество первых стартов  1186
